In [10]:
import numpy as np
import pandas as pd
import sklearn
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow_core.estimator import inputs
from tensorflow import feature_column
from tensorflow.keras import layers
from tensorflow.python.client import device_lib
tf.debugging.set_log_device_placement(True)


In [11]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('Label')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [37]:
train_vector_input = pd.read_csv("vector_input.csv")

In [23]:
# train_vector_input["Role"] = train_vector_input.Role.astype(str)
# train_vector_input["CompetitionWeek"] = train_vector_input.CompetitionWeek.astype(str)

In [38]:
role = feature_column.numeric_column("Role")


competition_week = feature_column.numeric_column("CompetitionWeek")


back_reps = feature_column.numeric_column("BackReps")


back_distance = feature_column.numeric_column("BackDistance")


back_split = feature_column.numeric_column("BackAverageSplit")


breast_reps = feature_column.numeric_column("BreastReps")


breast_distance = feature_column.numeric_column("BreastDistance")


breast_split = feature_column.numeric_column("BreastAverageSplit")


butterfly_reps = feature_column.numeric_column("ButterflyReps")


butterfly_distance = feature_column.numeric_column("ButterflyDistance")


butterfly_split = feature_column.numeric_column("ButterflyAverageSplit")


drill_reps = feature_column.numeric_column("DrillReps")


drill_distance = feature_column.numeric_column("DrillDistance")


drill_split = feature_column.numeric_column("DrillAverageSplit")


free_reps = feature_column.numeric_column("FreeReps")


free_distance = feature_column.numeric_column("FreeDistance")


free_split = feature_column.numeric_column("FreeAverageSplit")


im_reps = feature_column.numeric_column("ImReps")


im_distance = feature_column.numeric_column("ImDistance")


im_split = feature_column.numeric_column("ImAverageSplit")


kick_reps = feature_column.numeric_column("KickReps")


kick_distance = feature_column.numeric_column("KickDistance")


kick_split = feature_column.numeric_column("KickAverageSplit")


pull_reps = feature_column.numeric_column("PullPaddleReps")


pull_distance = feature_column.numeric_column("PullPaddleDistance")


pull_split = feature_column.numeric_column("PullPaddleAverageSplit")


total_distance = feature_column.numeric_column("TotalDistance")


In [ ]:
train, test = train_test_split(train_vector_input, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

In [ ]:
batch_size=32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [41]:
feature_columns = [role, competition_week, back_reps, back_distance,
                back_split, breast_reps, breast_distance, 
                breast_split, butterfly_reps, butterfly_distance,
                butterfly_split, drill_reps, drill_distance,
                drill_split, free_reps, free_distance,
                free_split, im_reps, im_distance, 
                im_split, kick_reps, kick_distance,
                kick_split, pull_reps, pull_distance,
                pull_split, total_distance
               ]

In [42]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [44]:
X_train, X_test = train_test_split(train_vector_input, test_size=0.1, random_state=101)

In [ ]:
X_test.dtypes

In [47]:
FEATURES = ['Role', 'CompetitionWeek', 'BackReps', 'BackDistance',
       'BackAverageSplit', 'BreastReps', 'BreastDistance',
       'BreastAverageSplit', 'ButterflyReps', 'ButterflyDistance',
       'ButterflyAverageSplit', 'DrillReps', 'DrillDistance',
       'DrillAverageSplit', 'FreeReps', 'FreeDistance', 'FreeAverageSplit',
       'ImReps', 'ImDistance', 'ImAverageSplit', 'KickReps', 'KickDistance',
       'KickAverageSplit', 'PullPaddleReps', 'PullPaddleDistance',
       'PullPaddleAverageSplit', 'TotalDistance']
LABEL= 'Label'

def get_input_fn(data_set, num_epochs=None, n_batch = 128, shuffle=True):
    return tf.compat.v1.estimator.inputs.pandas_input_fn(
       x=pd.DataFrame({k: data_set[k].values for k in FEATURES}),
       y = pd.Series(data_set[LABEL].values),
       batch_size=n_batch,   
       num_epochs=num_epochs,
       shuffle=shuffle)

In [ ]:
'''TensorFlow Estimator Linear Classifier Creation, training, and evaluation'''

model = tf.estimator.LinearClassifier(feature_columns=feature_columns,model_dir="ongoing/train4", n_classes=2)

model.train(get_input_fn(X_train, num_epochs=None, n_batch = 128, shuffle=False), steps=1000)
    
model.evaluate(get_input_fn(X_test, num_epochs=1, n_batch = 128, shuffle=False), steps=1000)

In [ ]:
model_1.evaluate(input_fn=get_input_fn(df_test_new, 
                                      num_epochs=1,
                                      n_batch = 128,
                                      shuffle=False),
                                      steps=1000)

In [ ]:
%time

'''Keras Sequential Model Creation and Training'''
with tf.device('/GPU:0'):
    model = tf.keras.Sequential([
      feature_layer,
      layers.Dense(128, activation='relu'),
      layers.Dense(128, activation='relu'),
      layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    model.fit(train_ds,
              validation_data=val_ds,
              epochs=5)
    
'''Check Keras Model Accuracy'''
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)


In [ ]:
'''Function to save Keras Model'''
model.save('keras\my_model')

'''Function to load Keras Model'''
# model = tf.keras.models.load_model('keras\my_model')
# model.summary()